<a href="https://colab.research.google.com/github/SubhashGovindharaj/Loan-Prediction/blob/main/loan_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('loan.csv')

In [ ]:
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


## data cleaning

In [ ]:
# missing values - Yes, imputed
# data right format - Yes
# structured data - Yes
# outliers - Yes
# duplicates - No

In [ ]:
df.isna().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [ ]:
df = df.dropna(subset = ['LoanAmount', 'Loan_Amount_Term'])

In [ ]:
df.isna().sum(), df.shape

(Loan_ID               0
 Gender               13
 Married               2
 Dependents           12
 Education             0
 Self_Employed        30
 ApplicantIncome       0
 CoapplicantIncome     0
 LoanAmount            0
 Loan_Amount_Term      0
 Credit_History       49
 Property_Area         0
 Loan_Status           0
 dtype: int64,
 (578, 13))

In [ ]:
for column in ['Gender', 'Married', 'Dependents', 'Self_Employed', 'Credit_History']:
  df[column] = df[column].fillna(df[column].mode()[0])

<ipython-input-332-d6cddae7306d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].fillna(df[column].mode()[0])


In [ ]:
df.isna().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [ ]:
df.dtypes

Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

In [ ]:
del df['Loan_ID']

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.shape

(578, 12)

In [ ]:
#only continous data we can correct outliers:
for col in ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']:
  iqr: float = df[col].quantile(0.75) - df[col].quantile(0.25)
  upper_threshold: float = df[col].quantile(0.75) + (1.5 * iqr)
  lower_threshold: float = df[col].quantile(0.25) - (1.5 * iqr)
  df[col] = df[col].clip(lower_threshold, upper_threshold)

## EDA + encoding

In [ ]:
df.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [ ]:
df.Loan_Status = df.Loan_Status.map({'Y':1, 'N':0})

### Gender

In [ ]:
df.groupby('Gender')['Loan_Status'].mean()

Gender
Female    0.669811
Male      0.703390
Name: Loan_Status, dtype: float64

In [ ]:
df['Gender'] = df['Gender'].map({'Female': 0, 'Male': 1})

### Married

In [ ]:
df.groupby('Married')['Loan_Status'].mean()

Married
No     0.636816
Yes    0.729443
Name: Loan_Status, dtype: float64

In [ ]:
df['Married'] = df['Married'].map({'No': 0, 'Yes': 1})

### Dependants

In [ ]:
df.groupby('Dependents')['Loan_Status'].mean()

Dependents
0     0.692537
1     0.649485
2     0.765306
3+    0.687500
Name: Loan_Status, dtype: float64

In [ ]:
del df['Dependents']

### Education

In [ ]:
df.groupby('Education')['Loan_Status'].mean()

Education
Graduate        0.713348
Not Graduate    0.636364
Name: Loan_Status, dtype: float64

In [ ]:
df['Education'] = df['Education'].map({'Not Graduate': 0, 'Graduate': 1})

### Self_Employed

In [ ]:
df.groupby('Self_Employed')['Loan_Status'].mean()

Self_Employed
No     0.700599
Yes    0.675325
Name: Loan_Status, dtype: float64

In [ ]:
df['Self_Employed'] = df['Self_Employed'].map({'No': 1, 'Yes': 0})

## ApplicantIncome

In [ ]:
df['ApplicantIncome']

1      4583.0
2      3000.0
3      2583.0
4      6000.0
5      5417.0
        ...  
609    2900.0
610    4106.0
611    8072.0
612    7583.0
613    4583.0
Name: ApplicantIncome, Length: 578, dtype: float64

In [ ]:
df.groupby('Loan_Status')['ApplicantIncome'].mean() # no pattern

Loan_Status
0    4639.154286
1    4615.129032
Name: ApplicantIncome, dtype: float64

In [ ]:
df['total_income'] = df['ApplicantIncome'] + df['CoapplicantIncome']
df.groupby('Loan_Status')['total_income'].mean()

Loan_Status
0    6061.920000
1    6041.272754
Name: total_income, dtype: float64

In [ ]:
df['ratio'] = df['LoanAmount'] * 100000 / df['total_income']
df.groupby('Loan_Status')['ratio'].mean()

Loan_Status
0    2544.361964
1    2363.604603
Name: ratio, dtype: float64

In [ ]:
del df['ApplicantIncome']
del df['CoapplicantIncome']
del df['total_income']

In [ ]:
df.columns

Index(['Gender', 'Married', 'Education', 'Self_Employed', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status',
       'ratio'],
      dtype='object')

In [ ]:
df.groupby('Loan_Status')['LoanAmount'].mean()

Loan_Status
0    143.920000
1    136.610422
Name: LoanAmount, dtype: float64

In [ ]:
df.groupby('Loan_Status')['Loan_Amount_Term'].mean()

Loan_Status
0    360.0
1    360.0
Name: Loan_Amount_Term, dtype: float64

In [ ]:
del df['Loan_Amount_Term']

In [ ]:
df.groupby('Loan_Status')['Credit_History'].mean()

Loan_Status
0    0.588571
1    0.982630
Name: Credit_History, dtype: float64

In [ ]:
# Convert 'Credit_History' column to numeric type
df['Credit_History'] = pd.to_numeric(df['Credit_History'], errors='coerce')

# Recalulate the mean
df.groupby('Loan_Status')['Credit_History'].mean()

Loan_Status
0    0.588571
1    0.982630
Name: Credit_History, dtype: float64

In [ ]:
df.groupby('Property_Area')['Loan_Status'].mean()

Property_Area
Rural        0.621302
Semiurban    0.768889
Urban        0.679348
Name: Loan_Status, dtype: float64

In [ ]:
df['Property_Area'] = df['Property_Area'].map({'Semiurban':2, 'Urban':1, 'Rural':0})

## Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.model_selection import train_test_split

X = df.loc[:, list(df.columns)[:-2] + ['ratio']]
y = df.loc[:, 'Loan_Status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 7)

In [ ]:
X_train.shape,X_test.shape

((433, 8), (145, 8))

## scaling - not mandatory

## Model

In [ ]:
#logistic regression
#Decision tree

#Random forest
#XG Boost

In [ ]:
df.Loan_Status.value_counts() / df.shape[0]

Loan_Status
1    0.697232
0    0.302768
Name: count, dtype: float64

### Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

model = LogisticRegression()
model.fit(X_train, y_train)
print(f"Test data accuracy : {model.score(X_test, y_test)}, F1-Score: {f1_score(model.predict(X_test), y_test)}")

Test data accuracy : 0.8137931034482758, F1-Score: 0.8820960698689957


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Decision tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score

for depth in [1,2,3,4,5,6,7,8,9,10]:
  model = DecisionTreeClassifier(max_depth = depth)
  cross_score = np.mean(cross_val_score(model, X_train, y_train, cv = 10))
  print(f"Depth: {depth}, Cross-Val Score: {cross_score}")


model = DecisionTreeClassifier(max_depth = 3)
model.fit(X_train, y_train)
print(f"Test data accuracy : {model.score(X_test, y_test)}, F1-Score: {f1_score(model.predict(X_test), y_test)}")

Depth: 1, Cross-Val Score: 0.8082452431289641
Depth: 2, Cross-Val Score: 0.8290697674418605
Depth: 3, Cross-Val Score: 0.8290697674418605
Depth: 4, Cross-Val Score: 0.8245771670190274
Depth: 5, Cross-Val Score: 0.8060253699788584
Depth: 6, Cross-Val Score: 0.7967758985200846
Depth: 7, Cross-Val Score: 0.7874735729386891
Depth: 8, Cross-Val Score: 0.7667019027484143
Depth: 9, Cross-Val Score: 0.7575052854122621
Depth: 10, Cross-Val Score: 0.7551268498942918
Test data accuracy : 0.8137931034482758, F1-Score: 0.8810572687224669


### Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(max_depth = 3, max_features = "sqrt")

for n in [10, 20, 50, 100, 200, 500, 1000]:
  model = RandomForestClassifier(n_estimators = n, max_depth = 3, max_features = "sqrt")
  cross_score = np.mean(cross_val_score(model, X_train, y_train, cv = 10))
  print(f"N: {n}, Cross-Val Score: {cross_score}")

N: 10, Cross-Val Score: 0.8038583509513743
N: 20, Cross-Val Score: 0.8267441860465116
N: 50, Cross-Val Score: 0.8267441860465116
N: 100, Cross-Val Score: 0.8267441860465116
N: 200, Cross-Val Score: 0.8267441860465116
N: 500, Cross-Val Score: 0.8290697674418605
N: 1000, Cross-Val Score: 0.8290697674418605


In [ ]:
model = RandomForestClassifier(n_estimators = 200, max_depth = 3, max_features = "sqrt")
model.fit(X_train, y_train)
print(f"Test data accuracy : {model.score(X_test, y_test)}, F1-Score: {f1_score(model.predict(X_test), y_test)}")

Test data accuracy : 0.8137931034482758, F1-Score: 0.8810572687224669


### XG-Boost

In [ ]:
from xgboost import XGBClassifier

#lambda - hyperparameter - lambda will take b/w 0-1

for lamb in [0.001, 0.002, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.011, 0.012, 0.013, 0.014, 0.015, 0.016, 0.017, 0.018, 0.019, 0.02, 0.05, 0.1, 0.2, 0.5, 1]:
  model = XGBClassifier(learning_rate = lamb)
  cross_score = np.mean(cross_val_score(model, X_train, y_train, cv = 10))
  print(f"Lambda: {lamb}, Cross-Val Score: {cross_score}")

Lambda: 0.001, Cross-Val Score: 0.6929175475687105
Lambda: 0.002, Cross-Val Score: 0.6929175475687105
Lambda: 0.005, Cross-Val Score: 0.8012684989429175
Lambda: 0.006, Cross-Val Score: 0.8174418604651162
Lambda: 0.007, Cross-Val Score: 0.8151162790697674
Lambda: 0.008, Cross-Val Score: 0.8174418604651162
Lambda: 0.009, Cross-Val Score: 0.8151162790697674
Lambda: 0.01, Cross-Val Score: 0.8220930232558139
Lambda: 0.011, Cross-Val Score: 0.8175475687103594
Lambda: 0.012, Cross-Val Score: 0.8175475687103594
Lambda: 0.013, Cross-Val Score: 0.8128964059196617
Lambda: 0.014, Cross-Val Score: 0.8106765327695561
Lambda: 0.015, Cross-Val Score: 0.8083509513742072
Lambda: 0.016, Cross-Val Score: 0.8106236786469345
Lambda: 0.017, Cross-Val Score: 0.8106236786469345
Lambda: 0.018, Cross-Val Score: 0.8106236786469345
Lambda: 0.019, Cross-Val Score: 0.8129492600422834
Lambda: 0.02, Cross-Val Score: 0.8129492600422834
Lambda: 0.05, Cross-Val Score: 0.8104122621564482
Lambda: 0.1, Cross-Val Score: 0.77

In [ ]:
model = XGBClassifier(learning_rate = 0.01)
model.fit(X_train, y_train)
print(f"Test data accuracy : {model.score(X_test, y_test)}, F1-Score: {f1_score(model.predict(X_test), y_test)}")

Test data accuracy : 0.7862068965517242, F1-Score: 0.8658008658008659
